In [1]:
!pip install -q tensorflow tensorflow-datasets

In [2]:
!pip install tensorflow.keras

In [3]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [4]:
dataset,info=tfds.load('plant_village',with_info=True,as_supervised=True)
teain_ds_full=dataset['train']
print('Number of classes: ',info.features['label'].num_classes)
print('Class names: ',info.features['label'].names)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/plant_village/incomplete.40426M_1.0.2/plant_village-train.tfrecord*...:   …

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.
Number of classes:  38
Class names:  ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_bligh

In [5]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

train_ds_full = dataset['train'].map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

total_examples = info.splits["train"].num_examples
train_size = int(0.8 * total_examples)
val_size = total_examples - train_size

train_ds = train_ds_full.take(train_size).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = train_ds_full.skip(train_size).take(val_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),

    Dense(38, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stop]
)

Epoch 1/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 762s 560ms/step - accuracy: 0.5413 - loss: 1.7589 - val_accuracy: 0.8171 - val_loss: 0.5914
Epoch 2/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 783s 576ms/step - accuracy: 0.8633 - loss: 0.4392 - val_accuracy: 0.8737 - val_loss: 0.4024
Epoch 3/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 760s 560ms/step - accuracy: 0.9236 - loss: 0.2367 - val_accuracy: 0.8814 - val_loss: 0.4003
Epoch 4/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 739s 544ms/step - accuracy: 0.9513 - loss: 0.1468 - val_accuracy: 0.8707 - val_loss: 0.4717
Epoch 5/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 722s 531ms/step - accuracy: 0.9666 - loss: 0.1007 - val_accuracy: 0.8970 - val_loss: 0.4230
Epoch 6/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 730s 537ms/step - accuracy: 0.9722 - loss: 0.0874 - val_accuracy: 0.8992 - val_loss: 0.4236


In [ ]:
from google.colab import files
files.download("model(7).h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>